# Lab3. 회귀

### Context
#### Linear Regression
+ Simple Linear Regression
+ Multiple Linear Regression

#### Machine Learning Algorithm Based Regression
+ Decision Tree Regression
+ RandomForest Regression
+ MLP Regression

#### Evaluation
+ R<sup>2</sup>
+ Adjusted R<sup>2</sup>

In [ ]:
import os
from os.path import join
import copy
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import sklearn

- Boston 주택 data를 불러온다.
- Boston dataset은 범죄율, 강의 인접 유무, 세금 등 13개의 변수를 가지고 있으며, 주택 가격을 label data로 가지고 있다.<br /><br />
- CRIM : 지역별 범죄 발생률
- ZN : 25,000평방피트를 초과하는 거주 지역의 비율
- INDUS : 비상업 지역 넓이 비율
- CHAS : 찰스강에 대한 더미 변수(강이ㅡ 경게에 위치한 경우 1, 아니면 0)
- NOX : 일산화질소 농도
- RM : 거주할 수 있는 방 갯수
- AGE : 1940년 이전에 건축된 소유 주택의 비율
- DIS : 5개 주요 고용센터까지의 가중 거리
- RAD : 고속도로 접근 용이도
- TAX : 10,000 달러당 재산세율
- PTRATIO : 지역의 교사와 학생 수 비율
- B : 지역의 흑인 거주 비율
- LSTAT : 하위 계층의 비율
- MEDV : 본인 소융의 주택 가격(중앙값)

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

In [ ]:
print(boston.DESCR)

In [ ]:
data = boston.data
label = boston.target    #주택가격
columns = boston.feature_names

In [ ]:
df = pd.DataFrame(data, columns = columns)
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

### Boston 주택 가격 data를 가지고 시각화를 해보자.

In [ ]:
import seaborn as sns

# 2개의 행과 4개의 열을 가진 subplots를 이용. axs는 4x2개의 ax를 가짐.
fig, axs = plt.subplots(figsize=(16,8) , ncols=4 , nrows=2)
lm_features = ['RM','ZN','INDUS','NOX','AGE','PTRATIO','LSTAT','RAD']
df['PRICE'] = label
for i , feature in enumerate(lm_features):
    row = int(i/4)
    col = i%4
    # Seaborn의 regplot을 이용해 산점도와 선형 회귀 직선을 함께 표현
    sns.regplot(x=feature , y='PRICE',data=df , ax=axs[row][col])

- 다른 칼럼보다 RM과 LSTAT의 PRICe 영향도가 가장 두드러지게 나타나는 것을 알 수 있다.
- RM(방갯수)은 양의 방향의 선형성이 가장 크다.
- 즉, 방의 크기가 클수록 가격이 증가하는 모습을 보인다.
- 또한 LSTAT(하위 계층의 비율)은 음의 방향의 선형성이 가장 크다.
- 즉, LSTAT이 적을 수록 PRICE가 증가하는 모습이 보인다.

# Regression
- Regression 분석이란, 두 변수 사이의 관계를 통계적 분석을 통해 방정식을 구하는 것이다. 
- 용어를 종속 변수, 독립 변수로 표현하면 이해하기 어려우니까 다음 수식에서의 y, x 로 표현한다. 
- 보통 여기서 표현하는 x는 원인을 제공한다고 하며, y는 결과를 파악한다고 한다.
- x는 독립변수(Independent Variable), 설명변수(Explanatory Variable)이라고 한다.
- y는 종속변수(Dependent Variable), 목적변수(Criterion Variable)이라고 한다.
- 종속변수는 결과를 나타내며, 우리가 알고 싶어 하는 값을 말한다.
- 그리고 독립변수는 종속변수를 움직일 수 있는 변수로 입력을 나타내며, 독립변수가 변하면서 종속변수가 어떻게 변하는지 알기 위한 변수이다.
- 예를 들면, 공부시간과 성적의 관계를 보면 성적이 공부시간에 영향을 미치지는 않지만, 공부시간이 성적에 영향을 미친다고 볼 수 있다.
- 이럴 때, 공부시간은 독립변수가 되며, 성적은 종속변수가 된다.
- 회귀분석은 크게 나누어 단순회귀(Simple Regression)과 다중회귀(Multiple Regression)으로 나눌 수 있다.
- 다음 그림과 같이 단순회귀는 독립변수 x가 한 개이며, 다중회귀는 독립변수 x가 두 개 이상으로 구성되며, 회귀식도 독립변수의 갯수만큼 단순회귀에 비해 복잡해 진다.

<img src = './img/Regression.png' alt='Regression' align='center' height=500 width=500 />

- 위의 그림의 단순회귀 방정식에 보면 독립변수가 2개인 것처럼 보이지만 변수의 갯수를 셀 때에는 상수항 x<sub>0</sub>은 포함시키지 않는 것이 일반적이다.
- 상수항 x<sub>0</sub>가 0이면 y 절편이 0이 되어 원점을 지나는 회귀직선이 되며, x<sub>0</sub>가 1이면 y 절편이 0이 아닌 a<sub>0</sub>나 b를 지나는 회귀직선이 된다.
- 회귀분석은 인구수, 회사의 이익 등 주어진 데이터에 대해 모델(회귀식)을 만든 후에 <b><u>예측</u></b>하는 곳에 주로 사용된다.

## Linear Regression
- 선형 회귀는 종속 변수와 한개 이상의 독립 변수와의 선형 상관 관계를 Modeling하는 회귀 분석 기법이다.

$$ y = wx + b$$
$$ y = w_0x_0 + w_1x_1 + w_2x_2 + .... w_nx_n + b$$
$$ w : 계수(가중치) $$
$$ b : 절편(편향) $$

- 간단하게 생각해보면 선형 회귀는 데이터가 분포되어 있는 공간에서 데이터를 가장 잘 표현하는 선을 하나 긋는다고 생각할 수 있다.
- 선형 회귀의 비용 함수는 다음과 같이 표현될 수 있다.

$$ Cost_{lr} = \sum_i{(y_i - \hat y_i)^2}$$
$$ \hat y_i = b + wx_i $$

- 결국 실제 참값 $y_i$와 회귀 모델이 출력한 $ \hat y $ 사이의 잔차의 제곱의 합을 최소화하는 w(계수)를 구하는 것이 목적이다. -> Least Square, 최소 제곱법
- 선형 회귀는 출력되는 y가 1개 또는 2개 이상인지의 유무에 따라 단변량, 다변량이라는 말이 붙는데, 이번 Lab에서는 출력값인 y가 1개(단변량)라고 가정한다.
- 또한, 입력으로 들어가는 x가 1개 또는 2개 이상인지의 유무에 따라 단순(Simple), 다중(Multiple)이라는 말이 붙는데, 이번 Lab에서는 단순, 다중 선형 회귀 분석에 대해 모두 학습한다.

#### 선형 회귀분석의 4가지 기본 가정
- 선형 회귀에는 4가지 가정이 필요하다.
- 맨 아래 참조 목록에 4가지 가정에 대해 잘 설명해준 페이지의 링크로 이론적인 설명은 대신하려고 한다.
1. 선형성
2. 독립성
3. 등분산성
4. 정규성

### 1. Simple Linear Regression
- 선형 회귀의 첫 번째로 x가 1개인 단순 회귀 분석에 대해 실습해 본다.
- x 변수로 'RM' 변수를, y 변수는 주택 가격으로 한다.
- Linear Regression은 Sklearn의 linear_model 패키지에 있다.

* Data를 train, test로 나누어 진행한다.
* sklearn의 model_selection 패키지에 있는 train_test_split 함수를 사용한다.
* train_test_split()은 학습과 test dataset를 분리해서 학습과 예측을 수행한다.

In [ ]:
del df

data = boston.data
df = pd.DataFrame(data, columns = columns)
from sklearn.model_selection import train_test_split
# test_size : 20% test data, 80% train data
# random_state : random의 seed값

x_train, x_test, y_train, y_test = train_test_split(df, label, test_size=0.2, random_state=2019)

#### 1) 모델 불러오기 및 정의하기

In [ ]:
from sklearn.linear_model import LinearRegression
sim_lr = LinearRegression()

#### 2) 모델 학습하기 (훈련 데이터)

In [ ]:
sim_lr.fit(x_train['RM'].values.reshape((-1, 1)), y_train)

#### 3) 결과 예측하기 (테스트 데이터)

In [ ]:
y_pred = sim_lr.predict(x_test['RM'].values.reshape((-1, 1)))

#### 4) 결과 살펴보기
- 일반적으로 선형회귀 R<sup>2</sup>를 평가 척도로 사용한다.
- R<sup>2</sup>값이 1에 가까울수록 회귀 모델이 데이터를 잘 표현한다는 것을 의미한다.

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
print('단순 선형 회귀, R2 : {:.4f}'.format(r2_score(y_test, y_pred)))

##### 0.1795이면 변수하나로는 제대로 예측하지 못한다는 의미

In [ ]:
line_x = np.linspace(np.min(x_test['RM']), np.max(x_test['RM']), 10)
line_y = sim_lr.predict(line_x.reshape((-1, 1)))
plt.scatter(x_test['RM'], y_test, s=10, c='black')
plt.plot(line_x, line_y, c = 'red')
plt.legend(['Regression line', 'Test data sample'], loc='upper left')

#### 회귀 모델의 계수 w, 절편 b 살펴보기
- 어떤 변수에 얼마 만큼의 가중치가 할당되고, 절편 값은 얼마나 할당되는지 살펴볼 수 있다.

In [ ]:
print('단순 선형 회귀, 계수(w) : {:.4f}, 절편(b) : {:.4f}'.format(sim_lr.coef_[0], sim_lr.intercept_))

### 2. Multiple Linear Regression
- 이번에는 x의 개수가 2개 이상인 다중 회귀 분석에 대해 학습한다.
- x 변수로 Boston dataset에 존재하는 모든 변수를 사용한다.

#### 1) 모델 불러오기 및 정의하기

In [ ]:
mul_lr = LinearRegression()

#### 2) 모델 학습하기 (훈련 데이터)

In [ ]:
mul_lr.fit(x_train, y_train)

#### 3) 결과 예측하기 (테스트 데이터)

In [ ]:
y_pred = mul_lr.predict(x_test)

#### 4) 결과 살펴보기
- 일반적으로 선형회귀 R<sup>2</sup>를 평가 척도로 사용한다.
- R<sup>2</sup>값이 1에 가까울수록 회귀 모델이 데이터를 잘 표현한다는 것을 의미한다.

In [ ]:
print('다중 선형 회귀, R2 : {:.4f}'.format(r2_score(y_test, y_pred)))

##### 0.6174이면 사용할 만한 데이터라는 의미

- Boston dataset이 가지고 있는 x 변수의 수는 13개, 그리고 주택 가격인 y 변수까지 합치면 14개의 변수이다.
- 이는 14개의 차원으로 표현된다는 의미이고, 사람은 최대 3차원까지만 인식할 수 있으므로 시각화를 하기에는 어려움이 있다.

#### 회귀 모델의 계수 w, 절편 b 살펴보기
- 어떤 변수에 얼마 만큼의 가중치가 할당되고, 절편 값은 얼마나 할당되는지 살펴볼 수 있다.

In [ ]:
print('다중 선형 회귀, 계수(w) : {}, 절편(b) : {:.4f}'.format(mul_lr.coef_, mul_lr.intercept_))

## Machine Learning Algorithm Based Regression
- 이번에는 Machine Learning Algorithm을 기반으로한 회귀 모델에 대해 알아보자.
- Sklearn이 지원하는 Machine Learning 기반 회귀 모델로는 Decision Tree, Random Forest, SVM(Support Vector Machine), MLP, AdaBoost, Gradient Boosting 등이 있다.
- 그 중 Decision Tree, Random Forest, SVM(Support Vector Machine), MLP 회귀 모델을 살펴본다.

### 1. Decision Tree Regressor
- Tree Model은 데이터의 불순도(impurity, Entropy)를 최소화 하는 방향으로 Tree를 분기하여 Model을 생성한다.
- 자세한 내용은 분류(Classification) Lab4에 나온다.
- 결정 트리 회귀 모델은 Sklearn의 tree 패키지에 있다. 

<img src = 'img/DecisionTree.png' alt='DecisionTree' />

#### 1) 모델 불러오기 및 정의하기

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt_regr = DecisionTreeRegressor(max_depth=5)

#### 2) 모델 학습하기 (훈련 데이터)

In [ ]:
dt_regr.fit(x_train['RM'].values.reshape((-1, 1)), y_train)

#### 3) 결과 예측하기 (테스트 데이터)

In [ ]:
y_pred = dt_regr.predict(x_test['RM'].values.reshape((-1, 1)))

#### 4) 결과 살펴보기
- 일반적으로 선형회귀 R<sup>2</sup>를 평가 척도로 사용한다.
- R<sup>2</sup>값이 1에 가까울수록 회귀 모델이 데이터를 잘 표현한다는 것을 의미한다.

In [ ]:
print('단순 결정 트리 회귀, R2 : {:.4f}'.format(r2_score(y_test, y_pred)))

In [ ]:
line_x = np.linspace(np.min(x_test['RM']), np.max(x_test['RM']), 10)
line_y = dt_regr.predict(line_x.reshape((-1, 1)))

plt.scatter(x_test['RM'].values.reshape((-1, 1)), y_test, c = 'black')
plt.plot(line_x, line_y, c = 'red')
plt.legend(['Regression line', 'Test data sample'], loc='upper left')

### 13개의 변수를 모두 사용해 결정 트리 회귀 모델을 사용하기

In [ ]:
dt_regre = DecisionTreeRegressor(max_depth=5)
dt_regre.fit(x_train, y_train)
y_pred = dt_regre.predict(x_test)
print('다중 결정 트리 회귀, R2 : {:.4f}'.format(r2_score(y_test, y_pred)))

### 2. Support Vector Machine Regressor
- SVM의 기본 개념은 결정 경계와 가장 가까운 데이터 샘플의 거리(Margin)을 최대화 하는 방식으로 모델을 조정하는 것이다.
- 자세한 내용은 역시 분류 Lab에서 다룬다.
- 서포트 벡터 머신 회귀 모델은 Sklearn의 svm 패키지에 있다.<br /><br /><br />
- SVM Demo(https://www.csie.ntu.edu.tw/~cjlin/libsvm/#nuandone)

<img src='img/SVM.jpg' alt='Support Vector Machine' />

#### 1) 모델 불러오기 및 정의하기

In [ ]:
from sklearn.svm import SVR
svm_regr = SVR()

#### 2) 모델 학습하기 (훈련 데이터)

In [ ]:
svm_regr.fit(x_train['RM'].values.reshape((-1, 1)), y_train)

#### 3) 결과 예측하기 (테스트 데이터)

In [ ]:
y_pred = svm_regr.predict(x_test['RM'].values.reshape((-1, 1)))

#### 4) 결과 살펴보기
- 일반적으로 선형회귀 R<sup>2</sup>를 평가 척도로 사용한다.
- R<sup>2</sup>값이 1에 가까울수록 회귀 모델이 데이터를 잘 표현한다는 것을 의미한다.

In [ ]:
print('단순 서포트 벡터 머신 회귀, R2 : {:.4f}'.format(r2_score(y_test, y_pred)))

In [ ]:
line_x = np.linspace(np.min(x_test['RM']), np.max(x_test['RM']), 100)
line_y = svm_regr.predict(line_x.reshape((-1, 1)))

plt.scatter(x_test['RM'], y_test, c = 'black')
plt.plot(line_x, line_y, c = 'red')
plt.legend(['Regression line', 'Test data sample'], loc='upper left')

### 13개의 변수를 모두 사용해 서포트 벡터 머신 회귀 모델을 사용하기

In [ ]:
svm_regr = SVR(C=20,)
svm_regr.fit(x_train, y_train)
y_pred = svm_regr.predict(x_test)
print('다중 서포트 벡터 머신 회귀, R2 : {:.4f}'.format(r2_score(y_test, y_pred)))

### 3. Multi Layer Perceptron Regressor
- Deep Learning의 기본 모델인 Neural Network를 기반으로 한 회귀 모델.
- 기본적으로 MLP라 하면, 입력층-은닉층-출력층 3개로 이루어진 Neural Network를 의미
- Perceptron : 뇌를 묘사한 Neural Network

<img src='img/neuran.png' alt='Neuran' width='550' height='700' /><br />
![Feedforword](./img/Feedforword.png)

#### 어떻게 Neural Network가 비선형 문제를 해결할 수 있을까?
- 은닉층에 존재하는 하나하나의 node는 기본 선형 회귀 model과 동일하게 $ wx + b $로 이루어져 있다. 
- 하지만 이런 선형 분리를 할 수 있는 model을 여러개를 모아 비선형 분리를 수행하는 것이 Neural Network 이다.

- MLP 회귀 모델은 Sklearn의 neural_network package에 있다.

#### 1) 모델 불러오기 및 정의하기

In [ ]:
from sklearn.neural_network import MLPRegressor
mlp_regr = MLPRegressor(solver='lbfgs')

#### 2) 모델 학습하기 (훈련 데이터)

In [ ]:
mlp_regr.fit(x_train['RM'].values.reshape((-1, 1)), y_train)

#### 3) 결과 예측하기 (테스트 데이터)

In [ ]:
y_pred = mlp_regr.predict(x_test['RM'].values.reshape((-1, 1)))

#### 4) 결과 살펴보기
- 일반적으로 선형회귀 R<sup>2</sup>를 평가 척도로 사용한다.
- R<sup>2</sup>값이 1에 가까울수록 회귀 모델이 데이터를 잘 표현한다는 것을 의미한다.

In [ ]:
print('단순 MLP 회귀, R2 : {:.4f}'.format(r2_score(y_test, y_pred)))

In [ ]:
line_x = np.linspace(np.min(x_test['RM']), np.max(x_test['RM']), 10)
line_y = mlp_regr.predict(line_x.reshape((-1, 1)))

plt.scatter(x_test['RM'], y_test, c = 'black')
plt.plot(line_x, line_y, c = 'red')
plt.legend(['Regression line', 'Test data sample'], loc='upper left')

### 13개의 변수를 모두 사용해 MLP 회귀 모델을 사용하기

In [ ]:
mlp_regr = MLPRegressor(hidden_layer_sizes=(50, ), activation='tanh', solver ='sgd', random_state=2019)
mlp_regr.fit(x_train, y_train)
y_pred = mlp_regr.predict(x_test)
print('다중 MLP 회귀, R2 : {:.4f}'.format(r2_score(y_test, y_pred)))

## Evaluation
### R<sup>2</sup>
- R<sup>2</sup>는 결정계수(coefficient of determination)이 표기로서, 회귀식에 의해 실제 데이터를 어느 정도 설명하는가를 알려주는 지표이다.
- 실제값과 예측값이 가지는 변동을 이용해서 이 지표를 계산한다.
- 변동이란 분산분석에서 나오는 정의로, 편차들의 제곱의 합을 의미한다.
- 회귀식이 실제값을 완벽하게 예측해서 잔차가 0이 되면, R<sup>2</sup>는 1로 최대값을 가지며
- 회귀식의 성능이 좋지 않아서 회귀식으로 설명할 수 있는 부분이 없다면, 잔차가 0이 되고 R<sup>2</sup>는 최소값 0을 가지게 된다.
- Scikit-Learn에서 지원하는 회귀 모델의 평가 방법으로는 R<sup>2</sup>가 있다.
- 학습한 회귀 모델이 얼마나 데이터를 잘 표현하는지에 대한 정도를 나타내는 통계적인 척도이며, 0 < R<sup>2</sup> < 1 범위의 값을 갖는다.
- 참고로, 선형회귀에서 최소제곱법으로 회귀식을 만들 경우, 상관계수의 제곱과 결정계수는 같다.

<img src = './img/R2.png' alt='R2' align='left' height=500 width=500 /> 

<div style='text-align:center;font-size:25pt'>잔차</div>
<img src='img/residual1.png' alt='Residual' width='500' height='500'/>

- SST(Total sum of squares) : 모든 개체가 가지는 총 변량
- SSR(Regression sum of squares) : 개체가 가지는 총 변량 중 회귀식으로 설명 가능한 변량
- SSE(Error sum of Squares) : 개체가 가지는 총 변량 중 회귀식으로 설명 불가능한 변량
- 잔차(residual) : 실제 값(Y)과 회귀식으로 예측한 값(y)의 차이

<img src='img/R2_1.png' alt='R2' />

$$ R^2 = 1 - {SSR \over SST} $$

* R<sup>2</sup> = 1, 모델이 데이터를 완벽하게 표현함 (Fits perfectly)
* R<sup>2</sup> = 0, 모델이 데이터를 전혀 표현하지 못함 (Does not explain anything)

### Adjust R<sup>2</sup>
- R<sup>2</sup>가 회귀 모델을 평가하기에 좋은 척도이긴 하지만 변수의 수가 증가하는 경우, R<sup>2</sup> 값은 모델 성능에 관계없이 값이 유지되거나 증가하게 된다.
- 위에서 계산한 결정계수에도 약점이 있는데, 변수의 갯수가 늘어날수록 예측 성능이 좋아지므로 결정계수는 1에 가까워지게 된다.
- Adjust R<sup>2</sup>는 이러한 문제를 해결하기 위해 나온 평가 척도이며, 변수의 수를 고려하여 R<sup>2</sup> 값을 계산한다.
- 따라서 변수가 추가되었을 때 추가된 변수가 모델의 성능에 영향을 주는지 확인할 수 있게 된다.

$$\bar R^2 = 1 - (1 - R^2){{n-1} \over {n - p - 1}} = 1 - {{SSR/df_e} \over {SST/df_t}} $$<br>
$$ df_e : 샘플의\ 자유도$$<br>
$$ df_t : 샘플\ 및\ x변수의\ 수를\ 고려한\ 자유도$$

- 아쉽게도 Adjust R<sup>2</sup>는 Sklearn에서 지원하지는 않는다.

### Reference
- 선형 회귀의 기본 가정 : https://kkokkilkon.tistory.com/175
- Wikipedia, Linear Regression : https://ko.wikipedia.org/wiki/선형_회귀
- Wikipedia, R-Square : https://en.wikipedia.org/wiki/Coefficient_of_determination
- Sklearn, Boston dataset : https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html
- Sklearn, Linear Regression : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
- Sklearn, Decision Tree Regression : https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
- Sklearn, Support Vector Machine Regression : https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
- Sklearn, MLP Regression : https://www.google.com/url?q=http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html&sa=U&ved=0ahUKEwjsntScreDhAhWMfbwKHWBXAbIQFggOMAU&client=internal-uds-cse&cx=016639176250731907682:tjtqbvtvij0&usg=AOvVaw3lJrRpqB9lWv_Fd98pGuJD
- Sklearn, R-Square : https://www.google.com/url?q=http://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html&sa=U&ved=0ahUKEwicgu3vueDhAhUI9LwKHeLDD3UQFggEMAA&client=internal-uds-cse&cx=016639176250731907682:tjtqbvtvij0&usg=AOvVaw3JYUuCpR-KNsPU189XgvWR